### 1. Discovery back model
Entender cómo se utiliza el modelo de ml subyacente para entrenar y hacer predicciones de series de tiempo

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.datasets import fetch_dataset
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

import numpy as np

### 1. Data

In [5]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0}
)

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data['y']
data = data.sort_index()

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)


In [6]:
data

datetime
1991-07-01    0.429795
1991-08-01    0.400906
1991-09-01    0.432159
1991-10-01    0.492543
1991-11-01    0.502369
                ...   
2008-02-01    0.761822
2008-03-01    0.649435
2008-04-01    0.827887
2008-05-01    0.816255
2008-06-01    0.762137
Freq: MS, Name: y, Length: 204, dtype: float64

### 2. Entrenar forecaster simple y obtener predicción h=1

In [7]:
from sklearn.linear_model import LinearRegression

In [30]:
forecaster = ForecasterAutoreg(
                 regressor = LinearRegression(),
                 lags      = 3
             )

In [31]:
forecaster.fit(y=data)

In [32]:
forecaster.predict(1)

2008-07-01    0.763618
Freq: MS, Name: pred, dtype: float64

### 3. Obtener el modelo subyacente y hacer predicción.
Comparar predicción de forecaster vs predicción hecha directamente con el modelo subyacente

In [35]:
# obtener el modelo subyacente dentro de la clase forecaster
model_regressor = forecaster.regressor
model_regressor

LinearRegression()

In [56]:
# generar instancia para predecir.
# SE ENTRENÓ CON 3 LAGS. Las features se deberían ordenar: [LAG1, LAG2, LAG3]
instance = np.flip(data.tail(3).values)
instance = np.expand_dims(instance, axis = 0)
instance

array([[0.762137, 0.816255, 0.827887]])

In [58]:
# predecir con el modelo subyacente creando mi instancia de entrada
model_regressor.predict(instance)

/opt/anaconda3/envs/data-science-python-3-10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([0.76361762])

### SE PUEDE OBSERVAR QUE ES EL MISMO VALOR. TAL COMO SE ESPERABA

### 4. Entrenar modelo sin utilizar habilitador forecaster
- Se debe generar todo el dataframe de entrenamiento X, y
- Entrenar el modelo
- Lo que debería pasar es que el resultado SEA EL MISMO. Al final la clase skforecast solo funcione como habilitador para hacer forecast recursivamente pero SI SE DESEARA CREAR TODO MANUALMENTE SE PODRÍA HACER SIN PROBLEMAS

In [86]:
### generar dataframe X
X = pd.DataFrame()
X['lag1'] = data.shift(1) # data lag 1
X['lag2'] = data.shift(2) # data lag 2
X['lag3'] = data.shift(3) # data lag 3
X = X.dropna()

In [87]:
### generar dataframe y
y = pd.DataFrame()
y['target'] = data
y = y.loc[X.index] # filtrar el datetime de datos de entrenamiento con lag no nulos

In [88]:
# entrenar
lr = LinearRegression()
lr.fit(X, y)

LinearRegression()

In [89]:
lr.score(X, y)

0.568814016278212

In [90]:
# predecir una observacion futuro - la misma utlizada en ejemplos previos
instance

array([[0.762137, 0.816255, 0.827887]])

In [91]:
lr.predict(instance)

/opt/anaconda3/envs/data-science-python-3-10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[0.76361762]])